In [ ]:
# https://www.kaggle.com/datasets/amanik000/gastrointestinal-disease-dataset
# This software analyzes a dataset linking 36 features of health data to various
# gastrointestinal disease states. The dataset can be found at
# https://www.kaggle.com/datasets/amanik000/gastrointestinal-disease-dataset
# This software could serve as the basis for a diagnostic tool to guide doctors
# in diagnosing gastrointestinal disorders.

**Demographic and Physical Attributes**

Age: Age of the individual (in years).

Gender: Biological gender of the individual (Male/Female/Other).

BMI: Body Mass Index – a measure of body fat based on height and weight.

Body_Weight: Weight of the individual (in kilograms or pounds).

**Health and Obesity Indicators**

Obesity_Status: Classification of individual as Obese, Overweight, or Normal based on BMI and other factors.

Ethnicity: Ethnic background (e.g., Caucasian, Asian, African American, etc.).

Family_History: Indicates whether there is a family history of related diseases (Yes/No).

**Biological and Genetic Markers**

Genetic_Markers: Presence of known genetic markers associated with disease risk.

Microbiome_Index: Quantitative index representing the state of the individual's gut microbiome.

**Autoimmune and Inflammatory Indicators**

Autoimmune_Disorders: Indicates the presence of known autoimmune disorders (Yes/No).

H_Pylori_Status: Status of Helicobacter pylori infection (Positive/Negative).

Fecal_Calprotectin: Inflammatory marker found in stool; high levels may indicate IBD.

Occult_Blood_Test: Results of fecal occult blood testing (Positive/Negative).

CRP_ESR: Combined result of C-Reactive Protein and Erythrocyte Sedimentation Rate – inflammation indicators.

**Diagnostic Results**

Endoscopy_Result: Findings from endoscopy (e.g., Normal, Inflammation, Ulcers).

Colonoscopy_Result: Findings from colonoscopy (e.g., Polyps, IBD, Normal).

Stool_Culture: Results indicating presence of bacterial, viral, or parasitic infections.

**Lifestyle and Diet**

Diet_Type: Type of diet followed (e.g., Vegetarian, Vegan, Omnivore).

Food_Intolerance: Known food intolerances (e.g., Lactose, Gluten).

Smoking_Status: Current smoking behavior (Smoker/Non-Smoker/Former Smoker).

Alcohol_Use: Frequency or status of alcohol consumption.

**Mental and Physical Health**

Stress_Level: Self-reported stress level (Low/Moderate/High).

Physical_Activity: Level of physical activity (Sedentary/Moderate/Active).

**Symptoms and GI Issues**

Abdominal_Pain: Presence of abdominal pain (Yes/No).

Bloating: Whether the individual experiences bloating (Yes/No).

Diarrhoea: Presence of diarrhea symptoms (Yes/No).

Constipation: Presence of constipation symptoms (Yes/No).

Rectal_Bleeding: Observation of rectal bleeding (Yes/No).

Appetite_Loss: Decreased appetite (Yes/No).

Weight_Loss: Unintentional weight loss (Yes/No).

Bowel_Habits: Summary or description of bowel movement characteristics.

Bowel_Movement_Frequency: Frequency of bowel movements (per day/week).

**Medication Usage**

NSAID_Use: Use of Non-Steroidal Anti-Inflammatory Drugs (Yes/No).

Antibiotic_Use: Recent use of antibiotics (Yes/No).

PPI_Use: Use of Proton Pump Inhibitors (Yes/No).

Medications: Other medications currently being taken.

**Target Variable**

Disease_Class: Final disease classification (e.g., Crohn’s Disease, Ulcerative Colitis, IBS, Healthy).

In [ ]:
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, DBSCAN
from scipy.linalg import svd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV, HalvingRandomSearchCV
import graphviz as gv
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import MultinomialNB


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# upload the dataset
df = pd.read_csv('./data/gastrointestinal_disease_dataset.csv')
num_duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicate_rows}")

In [ ]:
# examine the data
df.head()

In [ ]:
df.info()
# Note there are no Null fields
# BMI (Body Mass Index)
# The CRP/ESR ratio is a combination of measures of inflamation.
#   High Ratio (> 2): May point towards acute inflammation or bacterial infection.
#   Low Ratio (< 1): May suggest chronic inflammation or autoimmune conditions.
# PPI_Use (Proton pump inhibitors) are a class of medications that reduce the production of stomach acid.

In [ ]:
print(df.describe())

In [ ]:
print(df.nunique().to_frame(name='nunique'))
# Note all of the large numbers are for continuous data.
# Disease_Class is categorical but has only 6 categories. It is the target.

In [ ]:
df['Disease_Class'].value_counts()
# The target values are balanced.

In [ ]:
# simple view of number of unique values, min value, and max value
df_describe = df.describe()
df_nunique = df.nunique().to_frame(name='nunique')
combined_df = df_describe.T.merge(df_nunique, left_index=True, right_index=True, how='left')
print(combined_df[['nunique', 'min', 'max']])

In [ ]:
# Convert columns to numeric values
# One-hot encoding
df_dummies = pd.get_dummies(df, columns=['Ethnicity', 'Diet_Type', 'Bowel_Habits'], dtype='int8')

# Replace strings with numbers
df_dummies['Obesity_Status'] = df_dummies['Obesity_Status'].replace({'Underweight':-1, 'Normal':0, 'Overweight':1, 'Obese':2}).astype('int8')
df_dummies['Gender'] = df_dummies['Gender'].replace({'Male':0, 'Female':1}).astype('int8')

# Define the dtypes for multiple columns to reduce memory usage. The smallest available int is int8.
# Continuous features will be squared and cubed later so leaving as int64 or float64
dtype_mapping = {
    'Family_History': 'int8',
    'Autoimmune_Disorders': 'int8',
    'H_Pylori_Status': 'int8',
    'Occult_Blood_Test': 'int8',
    'Endoscopy_Result': 'int8',
    'Colonoscopy_Result': 'int8',
    'Stool_Culture': 'int8',
    'Food_Intolerance': 'int8',
    'Smoking_Status': 'int8',
    'Alcohol_Use': 'int8',
    'Abdominal_Pain': 'int8',
    'Bloating': 'int8',
    'Diarrhea': 'int8',
    'Constipation': 'int8',
    'Rectal_Bleeding': 'int8',
    'Appetite_Loss': 'int8',
    'Weight_Loss': 'int8',
    'NSAID_Use': 'int8',
    'Antibiotic_Use': 'int8',
    'PPI_Use': 'int8',
    'Medications': 'int8'
}

# Apply the dtypes
df = df_dummies.astype(dtype_mapping)

# Encode the target classes
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Disease_Class'] = le.fit_transform(df['Disease_Class'])

In [ ]:
df.info()

In [ ]:
df['Disease_Class'].head()
#le.inverse_transform(df['Disease_Class'])

In [ ]:
# Correlation Matrix
plt.figure(figsize=(64,64))
ax = sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".1f",  annot_kws={"size": 24})
plt.title('Correlation Matrix', fontsize=40)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
# The correlations of features with the target categories are very low.

In [ ]:
# feature engineering
# There is no effect of squaring or cubing the continuous features on the correlation with the target.
# I plotted the correlation matrices and could see no difference.
# Then I calculated the matrices and looked at the differences between them. There is none.
# I am proceeding with the non-engineered values for the continuous features after showing these calculations.
# You can uncomment the code for the plots below to see them.
# Temp dataframe
feats_eng = df[['Age', 'BMI', 'Body_Weight', 'Bowel_Movement_Frequency', 'CRP_ESR', 'Fecal_Calprotectin', 'Genetic_Markers', 'Microbiome_Index', 'Physical_Activity', 'Stress_Level']].copy()
feats_eng2 = feats_eng **2
feats_eng3 = feats_eng **3
feats_eng1 = pd.concat([feats_eng, df], axis=1)
feats_eng2 = pd.concat([feats_eng, df], axis=1)
feats_eng3 = pd.concat([feats_eng, df], axis=1)

# Correlation Matrix plot
#plt.figure(figsize=(16,16))
#ax = sns.heatmap(feats_eng1.corr(), annot=True, cmap='coolwarm', fmt=".2f",  annot_kws={"size": 10})
#plt.title('Correlation Matrix', fontsize=30)
#ax.tick_params(axis='x', labelsize=10)
#ax.tick_params(axis='y', labelsize=10)

feats_eng1.corr() - feats_eng3.corr()

In [ ]:
feats_eng1.corr() - feats_eng2.corr()

In [ ]:
feats_eng2.corr() - feats_eng3.corr()

In [ ]:
# Correlation Matrix with squared values
#plt.figure(figsize=(16,16))
#ax = sns.heatmap(feats_eng2.corr(), annot=True, cmap='coolwarm', fmt=".2f",  annot_kws={"size": 10})
#plt.title('Correlation Matrix with Features Squared', fontsize=30)
#ax.tick_params(axis='x', labelsize=10)
#ax.tick_params(axis='y', labelsize=10)

In [ ]:
# Correlation Matrix with cubed values
#plt.figure(figsize=(16,16))
#ax = sns.heatmap(feats_eng3.corr(), annot=True, cmap='coolwarm', fmt=".2f",  annot_kws={"size": 10})
#plt.title('Correlation Matrix with Features Cubed', fontsize=30)
#ax.tick_params(axis='x', labelsize=10)
#ax.tick_params(axis='y', labelsize=10)

In [ ]:
# Obesity_Status and BMI are highly correlated. Dropping Obesity_Status (range -1 - 2)
# because BMI has a wider range of values (range 16 - 40).
df = df.drop('Obesity_Status', axis=1)

# Split the data into train and test datasets
X = df.drop('Disease_Class', axis=1)
y = df['Disease_Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Clustering of disease class using K-means
X = np.array(X)
inertias = []
for i in range(2, 11):
  kmeans = KMeans(n_clusters=i, random_state=42).fit(X)
  inertias.append(kmeans.inertia_/1000000) # divide by 1000000 to make results easier to interpret
print(inertias)

In [ ]:
plt.plot(range(2, 11), inertias)
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia/1,000,000")
plt.title("Inertia vs. Number of Clusters")
# Number of Clusters = 4-5 are reasonable values

In [ ]:
# Clustering of disease classes using DBSCAN
# eps=19.0, min_samples=10 works
dbscan = DBSCAN(eps=19.0, min_samples=10).fit(X)
labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
unique_labels = set(labels)
n_clusters_ = len(unique_labels) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

# Although the Disease Class has 6 categories, it is possible that 4 categories will be more useful.

In [ ]:
# Pricipal Component Analysis using SVD to aid with understanding clusters
# Normalize and run PCA using SVD
def svd_norm(X):
    x_norm =(X - X.mean())/X.std()
    U, sigma, VT = svd(x_norm, full_matrices=False)
    Sigma = np.diag(sigma)
    return U, Sigma, VT
U, Sigma, VT = svd_norm(X_train)

# Plot x = r and y = Sigma
plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, len(Sigma) + 1), np.diag(Sigma), 'o-')
plt.title('Plot of Singular Values')
plt.xlabel('Singular Value Index (r)')
plt.ylabel('Singular Value ($\sigma$)')
plt.grid(True)
# r is very high. Proceeding with computation to be sure.

In [ ]:
# Plot x = r and y = Sigma
SigSum = []
for i in range(len(Sigma)):
  SigSum.append(np.diag(Sigma)[:i].sum())

plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, len(Sigma) + 1), SigSum/SigSum[-1], 'o-')
plt.title('Plot of Singular Values')
plt.xlabel('Singular Value Index (r)')
plt.ylabel('% Singular Value (%$\sigma$)')
plt.xticks([5, 10, 15, 20, 25, 30, 35, 40, 45])
plt.yticks([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
plt.grid(True)

# r is about 38 singular values to predict the target with 90% accuracy.
# This means virtually all of the features will be needed to categorize these data.

In [ ]:
# Classification
# Trying the various classification algorithms.
# Note that linear regression is not a good choice because the target has classes not a gradient.

# KNN: classification, new data compared to all training data is expensive.
# We cannot eliminate more than a few features. Therefore not using KNN.
start_time = time.time()
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")
# I estimate it will take about 2 hours wall time to tune KNN on this dataset.

In [ ]:
# Logistic Regression is best for 2 classes. These data have 4 - 6 classes.
# Not using Logistic Regression.
# Decision Tree
params = {
    'max_depth': [None, 1, 2, 3],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': [1, 2, 5]
}

# GridSearchCV
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(),
                           param_grid=params,
                           cv=5,
                           scoring='accuracy')
grid_search.fit(X_train, y_train)
print('GridSearchCV best score: ', grid_search.best_score_)
print(grid_search.best_params_)

# RandomizedSearchCV
grid_search = RandomizedSearchCV(estimator=DecisionTreeClassifier(),
                           param_distributions=params,
                           cv=5,
                           scoring='accuracy')
grid_search.fit(X_train, y_train)
print('RandomizedGridSearchCV best score: ', grid_search.best_score_)
print(grid_search.best_params_)

# HalvingGridSearchCV
grid_search = HalvingGridSearchCV(estimator=DecisionTreeClassifier(),
                           param_grid=params,
                           cv=5,
                           scoring='accuracy')
grid_search.fit(X_train, y_train)
print('HalvingGridSearchCV best score: ', grid_search.best_score_)
print(grid_search.best_params_)

# HalvingRandomSearchCV
grid_search = HalvingRandomSearchCV(estimator=DecisionTreeClassifier(),
                           param_distributions=params,
                           cv=5,
                           scoring='accuracy')
grid_search.fit(X_train, y_train)
print('HalvingRandomSearchCV best score: ', grid_search.best_score_)
print(grid_search.best_params_)

# mean_fit_time was not an issue.
# GridSearchCV gave the best score with DecisionTreeClassifier

In [ ]:
# GridSearchCV running DecisionTreeClassifier to tune min_samples_split.
# Note that HalvingRandomSearchCV sometimes gives a better score than
# GridSearchCV but not always.
params = {
    'max_depth': [1],
    'min_samples_split': [1, 2, 3, 4],
    'criterion': ['entropy'],
    'min_samples_leaf': [1]
}

# GridSearchCV
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(),
                           param_grid=params,
                           cv=5,
                           scoring='accuracy')
grid_search.fit(X_train, y_train)
print('GridSearchCV best score: ', grid_search.best_score_)
print(grid_search.best_params_)

In [ ]:
# Trying decision trees
decision_tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=1, min_samples_leaf=1, min_samples_split=2)
decision_tree_model.fit(X_train, y_train)
print(decision_tree_model.score(X_test, y_test))

In [ ]:
dot_data = tree.export_graphviz(decision_tree_model,
                                out_file=None,
                                feature_names=X_train.columns,
                                class_names=['Blood in stool', 'Abdominal cramps or pain', 'Nausea or vomiting', 'Unexplained weight loss', 'Bloating', 'Diarrhea or constipation'],
                                filled=True,
                                rounded=True)
graph = gv.Source(dot_data)
graph.render(format="png", filename="GI_tree")
graph
# This graph has the best accuracy score but is not useful for diagnosisng patients.
# It fails to diagnose the other 4 categories of the target class.
# Based on the clustering, we expect 4 categories. This will require 2 splits.
# Based on the number of target categories, we expect 3 splits.
# Trying 2 and 3 splits next.

In [ ]:
# 2 splits
decision_tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=2, min_samples_leaf=1, min_samples_split=2)
decision_tree_model.fit(X_train, y_train)
print(decision_tree_model.score(X_test, y_test))
# This gives leaves of 4 of the 6 classes.

In [ ]:
dot_data = tree.export_graphviz(decision_tree_model,
                                out_file=None,
                                feature_names=X_train.columns,
                                class_names=['Blood in stool', 'Abdominal cramps or pain', 'Nausea or vomiting', 'Unexplained weight loss', 'Bloating', 'Diarrhea or constipation'],
                                filled=True,
                                rounded=True)
graph = gv.Source(dot_data)
graph.render(format="png", filename="GI_tree")
graph

In [ ]:
# 3 splits
decision_tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=1, min_samples_split=2)
decision_tree_model.fit(X_train, y_train)
print(decision_tree_model.score(X_test, y_test))
# This gives leaves of 5 of the 6 classes. The numbers are not balanced but are balanced in X.

In [ ]:
dot_data = tree.export_graphviz(decision_tree_model,
                                out_file=None,
                                feature_names=X_train.columns,
                                class_names=['Blood in stool', 'Abdominal cramps or pain', 'Nausea or vomiting', 'Unexplained weight loss', 'Bloating', 'Diarrhea or constipation'],
                                filled=True,
                                rounded=True)
graph = gv.Source(dot_data)
graph.render(format="png", filename="GI_tree")
graph

In [ ]:
# HalvingRandomSearchCV running DecisionTreeClassifier to tune min_samples_split.
# Note that HalvingRandomSearchCV sometimes gives a better score than
# GridSearchCV but not always.
params = {
    'max_depth': [None],
    'min_samples_split': [6,7,8,9,10,11,12,13,14,15],
    'criterion': ['entropy'],
    'min_samples_leaf': [2,3,4]
}

# HalvingRandomSearchCV
grid_search = HalvingRandomSearchCV(estimator=DecisionTreeClassifier(),
                           param_distributions=params,
                           cv=5,
                           scoring='accuracy')
grid_search.fit(X_train, y_train)
print('GridSearchCV best score: ', grid_search.best_score_)
print(grid_search.best_params_)
# Ran repeatedly. Best result was
# GridSearchCV best score:  0.1761399686580285
# {'min_samples_split': 6, 'min_samples_leaf': 3, 'max_depth': None, 'criterion': 'entropy'}

In [ ]:
decision_tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=None, min_samples_leaf=3, min_samples_split=6)
decision_tree_model.fit(X_train, y_train)
print(decision_tree_model.score(X_test, y_test))

In [ ]:
# Produces a huge tree. Run at your own risk!
#dot_data = tree.export_graphviz(decision_tree_model,
#                                out_file=None,
#                                feature_names=X_train.columns,
#                                class_names=['Blood in stool', 'Abdominal cramps or pain', 'Nausea or vomiting', 'Unexplained weight loss', 'Bloating', 'Diarrhea or constipation'],
#                                filled=True,
#                                rounded=True)
#graph = gv.Source(dot_data)
#graph.render(format="png", filename="GI_tree")
#graph

In [ ]:
print("GraphViz version is from 2006: ", gv.version())
# There is a very old version of GraphViz on Colab. I was unable to find documentation for it. I wanted to change the size of the image.
# The size of of an unlimited depth decision tree is non-functional for these data.
# Because there are a large number of weak features bagging and specifically Random Forest are good algorithms to try.

In [ ]:
X

In [ ]:
# Random Forest
# Using all 45 columns as max_features because almost all needed to reach 90% accuracy.
# Takes 10 minutes wall time to run
model = RandomForestClassifier(n_estimators=1000, max_features=45, oob_score=True)
model.fit(X, y)
print(model.oob_score_)
# Still getting only a very weak signal out of these data.

In [ ]:
# Tuning max_features for RandomForestClassifier
# Takes about 15 minutes to run
#params = {'max_features': [45, 44, 43, 42, 41, 40],
#          'n_estimators': [100],
#          'oob_score': [True]}
#grid_search = GridSearchCV(estimator=RandomForestClassifier(),
#                           param_grid=params,
#                           cv=5,
#                           scoring='accuracy')
#grid_search.fit(X_train, y_train)
#print('GridSearchCV best score: ', grid_search.best_score_)
#print(grid_search.best_params_)

# Output
# GridSearchCV best score:  0.16737558251255585
# {'max_features': 44, 'n_estimators': 100, 'oob_score': True}

In [ ]:
# Running RandomForestClassifier with max_features tuned
# Takes about 6 minutes wall time to run
model = RandomForestClassifier(n_estimators=1000, max_features=44, oob_score=True)
model.fit(X, y)
print(model.oob_score_)

In [ ]:
# Pairplot out of desperation. This plots the continuous variables to see if any relationships jump out.
# I don't see any reason to use SVM to separate clusters because I don't see any complex relationships.
sns.pairplot(feats_eng)
# Apart from the expected relationship between weight and BMI, no relatioinships are visible.
# This is not surprising considering the correlation coefficients.

In [ ]:
# Linear Regression
lin_reg = LinearRegression(fit_intercept=False).fit(X, y)
y_pred = lin_reg.predict(X_test)
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))
# Meh

In [ ]:
# Naive Bayes
naive_bayes = MultinomialNB(alpha=1.0, fit_prior=False).fit(X_train, y_train)
print(naive_bayes.feature_log_prob_)
# This is too simple for the given problem.
# The numbers for each disease type (row) indicate which feature is most likely to be the cause
# The problem is we know that it will take almost all the features to predict the disease type.
# This is predicting the feature from the disease type instead of the disease type from the combined features.

In [ ]:
# Neural Nets

In [ ]:
# Gradient descent

In [ ]:
# Evaluate all the different models

In [ ]:
# NLP?

In [ ]:
# Recommendation System?